In [309]:
import array as arr
import csv
import queue    # see https://docs.python.org/3/library/queue.html#module-queue
import math
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
# from matplotlib.collections import Poly3DCollection # https://stackoverflow.com/questions/10599942/drawing-a-rectangle-or-bar-between-two-points-in-a-3d-scatter-plot-in-python-and
# and https://stackoverflow.com/questions/37585340/plotting-3d-polygons-in-python-3
import numpy as np
from trajectory import * # import the trapezoidal trajectory
import time

# interesting/useful websites
# talks about implementing robot simulator: https://www.toptal.com/robotics/programming-a-robot-an-introductory-tutorial  
# said robot simulator code: https://github.com/nmccrea/sobot-rimulator/blob/v1.0.0/models/supervisor.py

In [310]:
def step(t, dt):
    t = t + dt
    return t

In [311]:
def drawRect(q_curr, width_v, length_v):
    
    xl  = q_curr[0] + width_v / 2  # left x coordinate
    xr  = q_curr[0] - width_v / 2  # right x coordinate
    
    yf  = q_curr[1] - length_v / 2 # front
    yb  = q_curr[1] + length_v / 2 # back
    
    print("x-loc rectangle,", x, "y-loc rectangle,", y)
    
    verts = []

    xs = [xr, xl, xl, xr, xr] # each box has 4 vertices, give it 5 to close it, these are the x coordinates
    ys = [yf, yf, yb, yb, yf]  # each box has 4 vertices, give it 5 to close it, these are the y coordinates

    verts = [list(zip(xs, ys))]
       
    poly = PolyCollection(verts, facecolors="limegreen", closed = False)
    
    z_axis = [0, 0, 0, 0, 0]
    ax.add_collection3d(poly, zs=[z_axis[0]] * len(verts), zdir='z') # in the "z" just use the same coordinates

In [312]:
def vehicleStep(q_curr, v, dt):
    # for now it's constant velocity  
    q_new = np.array([q_curr[0], q_curr[1] + v * dt])
    return q_new

In [313]:
class camera(object):
    def __init__(self):
        ######################## MODULES ########################
        self.indexVisFruit = queue.SimpleQueue()
        
        
        #################### CLASS VARIABLES ####################
        self.width = 1         # width of camera view, in m
        self.i_lastFruit = 0   # last visited fruit index
#         self.indexVisFruit = []
        
        
    ######################## FUNCTIONS ########################
    def cameraStep(self, end_row, q_curr, sortedFruit):
        # centered (for now) on the vehicle, perpendicular to data's y-axis
        view_min = q_curr - self.width / 2
        view_max = q_curr + self.width / 2

        # fix end constraints of beginning and end 
        if view_min < 0:
            view_min = 0

        if view_max > end_row:
            view_max = end_row

        i = self.i_lastFruit
#         indexVisFruit = []
#         print("index of fruit:", i)

        # look through sorted (around y-axis) array to find all "visible fruit"
        while(True):
            if sortedFruit[1,i] > view_min and sortedFruit[1,i] < view_max:
#                 self.indexVisFruit.append(int(i))
                self.indexVisFruit.put(i)
                
            elif sortedFruit[1,i] > view_max:
                break
            i = i + 1
            
        self.i_lastFruit = i

#         return self.indexVisFruit

In [314]:
def schedule(indexVisFruit):
    # start simple by just popping out the next value (see if we can do this)
    i_goal = indexVisFruit.get()
#     print(i_goal)

In [315]:
def armStep(goal, q_v, v_v, q_curr, length, width_af, height_af, v_max, a_max, d_max, dt):
    # AAAALLLLL THE AAARMMMSSSSSSSSS
    # q_curr is the arm's location within it's frame => where is the frame in respect to the vehicle?
    # q_v is the vehicle's location
    
    # get trapezoidal times
    arm1_y.noJerkProfile(arm1_y.q0, goal[0], arm1_y.v0, v_max, a_max, d_max)  
    arm1_z.noJerkProfile(arm1_z.q0, goal[1], arm1_z.v0, v_max, a_max, d_max)  
    
#     print("Y-axis, Ta:", arm1_y.Ta, "Tv:", arm1_y.Tv, "Td:", arm1_y.Td)
#     print("Z-axis, Ta:", arm1_z.Ta, "Tv:", arm1_z.Tv, "Td:", arm1_z.Td)
    
    
#     q_a = q_curr + q_v  # center the arm on the vehicle's frame for now
    

In [316]:
# open CSV file and populate fruits on grid
x_fr = []
y_fr = []
z_fr = []

# with open('Applestotheleft.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#     for row in spamreader:
#         x_fr.append(float(row[0]))
#         y_fr.append(float(row[1]))
#         z_fr.append(float(row[2]))
        
with open('Applestotheright.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        x_fr.append(float(row[0]))
        y_fr.append(float(row[1]))
        z_fr.append(float(row[2]))

x_fruit = np.array(x_fr)
y_fruit = np.array(y_fr)
z_fruit = np.array(z_fr)

# need a matrix to sort x, y, and z based on the y-axis (to know what fruit show up earlier)
fruit = np.stack([x_fr, y_fr, z_fruit])

rowIndex = 1 # sort based on y-axis
sortedFruit = fruit[ :, fruit[rowIndex].argsort()]


In [317]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(sortedFruit[0,:], sortedFruit[1,:], sortedFruit[2,:], c='r', marker='o')

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# plt.show()

In [318]:
# init environment
noise = 0     # for when it becomes stochastic
end_row = 15  # end of the row
t = []         # 'global' time
t_new = 0
dt = 0.1      # 'global' time step size
runs = 0

In [319]:
# init vehicle
width_v = 1      # vehicle width
length_v = 5  # vehicle length
arms = 1         # set number of arms on robot
q_y  = []     
q_x  = []
q_v = np.array([5.5,9]) # starting location
v_v = 0.1        # in m, constant velocity

In [320]:
# init camera
picture = camera()

In [321]:
# init scheduler
goal = np.zeros(3) # does ithis go here?

In [322]:
# init arms
length    = 0.5    # how far can it go into trees, in m
width_af  = 2.5    # size of enclosing frame, width in m
height_af = 2      # size of enclosing frame, height in m

# values for trapezoid trajectory (parameters to play with...)
v_max     = 1
a_max     = 10 
d_max     = 10

q_a = np.array([0,0]) # location of arm in x, y? => need to figure out how to set these when there are >1 arms

# init trajectory module
arm1_y = Trajectory()
arm1_y.adjInit(0, 0)

arm1_z = Trajectory()
arm1_z.adjInit(0, 0)

In [323]:
##### while loooop!
while(q_v[1] < end_row):
    # env.step
    t_new = step(t_new, dt)
    t.append(float(t_new))
    
    
    # vehicle.step 
    q_v = vehicleStep(q_v, v_v, dt) # calculate "instantaneous" location
    q_x.append(float(q_v[0]))  # to plot it out later
    q_y.append(float(q_v[1]))
    
    # env.obs / camera... FLASH!
    picture.cameraStep(end_row, q_v[1], sortedFruit) # resulting index/coordinates in world frame  
    
    if (noise == 1):
        # add some noissse! (to the fruit locations)
        noise = 0 # but not right now
    
    # schedule
    if not picture.indexVisFruit.empty():
        i_f = picture.indexVisFruit.get()
#         print(picture.indexVisFruit.qsize())
        goal = sortedFruit[:,i_f]
#         print(goal)
        # need to add a check to see if the machine can still get it?
    else:
        goal = ([0,0,0])
    

    for arm in range(0,arms):
        # arm[i].step (in y and z for now)
        armStep(goal, q_v[1], v_v, q_a, length, width_af, height_af, v_max, a_max, d_max, dt)

        # arm[i].action?fruit?something?
        i = 1
        
    ## plot the vehicle's movement (add other parts as they get finished)    
#     if (runs % 20 == 0):
#         plt.show()

#         fig = plt.figure()
#         ax = fig.add_subplot(111, projection='3d')
#         plt.ion()   

#         fig.show()
#         fig.canvas.draw()

#         ax.clear()
#         ax.scatter(sortedFruit[0,:], sortedFruit[1,:], sortedFruit[2,:], c='r', marker='o') 
        
#         drawRect(q_v, width_v, length_v) # try and draw the vehicle moving along the orchard
                
#         ax.plot(q_x,q_y, 0)
        
#         ax.set_xlim(4, 12)
#         ax.set_ylim(q[0], end_row+3)
        
#         ax.set_xlabel('X Axis')
#         ax.set_ylabel('Y Axis')
#         ax.set_zlabel('Z Axis')
        
#         fig.canvas.draw()
#         time.sleep(0.25)
        
#     runs+=1
    

        
        

In the function!
0.0 7.9985286833 0.0
In the function!
0.0 8.5129762637 0.0
In the function!
0.0 9.333058669 0.0
In the function!
0.0 8.520953055 0.0
In the function!
0.0 8.4667118565 0.0
In the function!
0.0 8.5313880608 0.0
In the function!
0.0 6.9728310576 0.0
In the function!
0.0 8.5318300425 0.0
In the function!
0.0 8.8355059867 0.0
In the function!
0.0 8.5328606554 0.0
In the function!
0.0 8.0151685485 0.0
In the function!
0.0 8.5566864005 0.0
In the function!
0.0 7.4952996361 0.0
In the function!
0.0 8.558201391 0.0
In the function!
0.0 7.3819330374 0.0
In the function!
0.0 8.572004075 0.0
In the function!
0.0 8.0655797807 0.0
In the function!
0.0 8.5897312792 0.0
In the function!
0.0 8.1115651567 0.0
In the function!
0.0 8.6200564025 0.0
In the function!
0.0 8.5941126744 0.0
In the function!
0.0 8.6275245988 0.0
In the function!
0.0 8.5177589655 0.0
In the function!
0.0 8.6294386432 0.0
In the function!
0.0 6.9797056675 0.0
In the function!
0.0 8.6386615628 0.0
In the function!

In the function!
0.0 0 0
In the function!
0.0 8.8190390845 0
In the function!
0.0 11.7204793947 0
In the function!
0.0 8.4507430051 0
In the function!
0.0 11.7308591763 0
In the function!
0.0 8.9626737675 0
In the function!
0.0 11.7312676793 0
In the function!
0.0 9.1219701113 0
In the function!
0.0 11.7467979282 0
In the function!
0.0 8.1910831996 0
In the function!
0.0 11.7573257671 0
In the function!
0.0 8.4353655695 0
In the function!
0.0 11.7637952335 0
In the function!
0.0 8.650867249 0
In the function!
0.0 11.7718383878 0
In the function!
0.0 7.9811020368 0
In the function!
0.0 11.7728874296 0
In the function!
0.0 8.9260890299 0
In the function!
0.0 11.7837420458 0
In the function!
0.0 8.0014659565 0
In the function!
0.0 11.8009528641 0
In the function!
0.0 7.8885322033 0
In the function!
0.0 11.8253689299 0
In the function!
0.0 7.5277779734 0
In the function!
0.0 11.8289239325 0
In the function!
0.0 8.0928385344 0
In the function!
0.0 11.833148009 0
In the function!
0.0 8.31680

0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
0.0 0 0
In the function!
